## Setting
- master1_cleansing_map_before.csv 을 이용해 추가 클린징 진행
- 위 파일 배포자 참고▼   https://github.com/eeuunnzzyy/eeuunnzzyy/blob/main/The%20Transportation%20Vulnerable%20Project/000%20Dailydata%20Cleansing%20%26%20EDA.ipynb

In [384]:
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
import glob

In [385]:
paths = glob.glob('./data/*.csv')
paths

['./data/master_cleansing_space_time.csv',
 './data/master.csv',
 './data/master_cleansing_before_diff_은지.csv',
 './data/master1_cleansing_map_before.csv']

In [386]:
paths[2]

'./data/master_cleansing_before_diff_은지.csv'

In [387]:
df = pd.read_csv(paths[2])
df.head()

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동


In [388]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407553 entries, 0 to 407552
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   no         407553 non-null  int64 
 1   cartype    407553 non-null  object
 2   예정시간       407553 non-null  object
 3   배차시간       407553 non-null  object
 4   승차시간       407553 non-null  object
 5   startpos1  407553 non-null  object
 6   startpos2  407553 non-null  object
 7   endpos1    407553 non-null  object
 8   endpos2    407553 non-null  object
dtypes: int64(1), object(8)
memory usage: 28.0+ MB


In [389]:
df.shape

(407553, 9)

## 데이터셋 시계열 적재오류 점검
- master1_cleansing_map_before.csv

- 시간데이터 체크list: (예상시간< 배차시간 < 승차시간) 
1. 위 날짜가 역순인경우 존재
2. 밤12시에서 새벽 1시로 넘어갈때 일(day)가 바뀌지 않은경우 많음

In [390]:
df['예정시간'].sort_values(ascending = False)

406845    2022-06-25 08:20:00
406844    2022-06-25 08:19:49
406843    2022-06-25 08:19:33
406842    2022-06-25 08:18:46
406841    2022-06-25 08:18:25
                 ...         
4484      2018-01-01 00:25:00
4483      2018-01-01 00:21:00
4482      2018-01-01 00:08:00
4481      2018-01-01 00:04:00
4480      2018-01-01 00:03:00
Name: 예정시간, Length: 407553, dtype: object

In [391]:
df['배차시간'].sort_values(ascending = False)

406845    2022-06-25 08:33:47
406844    2022-06-25 08:29:39
406841    2022-06-25 08:28:01
406842    2022-06-25 08:26:22
406843    2022-06-25 08:26:21
                 ...         
4488      2018-01-01 02:47:07
4487      2018-01-01 02:42:20
4482      2018-01-01 02:27:53
4481      2018-01-01 02:24:19
4480      2018-01-01 02:19:23
Name: 배차시간, Length: 407553, dtype: object

In [392]:
df['승차시간'].sort_values(ascending = False)

406815    2022-06-25 09:00:04
406811    2022-06-25 08:58:29
406813    2022-06-25 08:55:10
406812    2022-06-25 08:49:03
406814    2022-06-25 08:48:00
                 ...         
4487      2018-01-01 03:15:58
4488      2018-01-01 03:01:41
4480      2018-01-01 02:48:48
4482      2018-01-01 02:48:29
4481      2018-01-01 02:46:04
Name: 승차시간, Length: 407553, dtype: object

※ 이상하다... 24시간제가 아니네 은지언니 물어보기

In [422]:
df["예정시간"]=pd.to_datetime(df["예정시간"])
df["배차시간"]=pd.to_datetime(df["배차시간"])
df["승차시간"]=pd.to_datetime(df["승차시간"])

In [423]:
# 역순 점검 예정-배차_역순, 배차-승차_역순.  2번 Test1
역순 =[]

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    time_3 = df['승차시간'][i]
    
    if time_1 > time_2: # datetime 끼리 비교 가능  
        역순.append(1)
    elif time_2 > time_3: 
        역순.append(1)
    else:
        역순.append(0)
        

In [427]:
# 역순 점검 예정-배차_역순, 배차-승차_역순.  2번 Test2
역순 =[]

for i in range(len(df)):
    time_2 = df['배차시간'][i]
    time_3 = df['승차시간'][i]
    
    if time_2 > time_3: 
        역순.append(1)
    else:
        역순.append(0)

In [428]:
df['역순'] = 역순

In [429]:
aa = df[df['역순']==1] # 배차시간>승차시간인 case : 407,553 중 51,405행  

In [430]:
aa.to_csv('master2_timediff_역순점검.csv', index = False, encoding = 'utf-8')

## 1. 3가지 차이 시간 칼럼 생성

In [393]:
import time
import datetime

### '배차-예정'

In [394]:
%%time

배차예정_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['예정시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['배차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    배차예정_val.append(time_interval)
    
##날짜와 시간(datetime)을 문자열로 출력하려면 strftime
##날짜와 시간 형식의 문자열을 datetime으로 변환하려면 strptime을 사용하면 된다

CPU times: user 8.18 s, sys: 23.7 ms, total: 8.21 s
Wall time: 8.21 s


In [395]:
df['배차-예정'] = 배차예정_val

### '승차-배차'

In [396]:
%%time

승차배차_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['배차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['승차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    승차배차_val.append(time_interval)

CPU times: user 8.19 s, sys: 19.3 ms, total: 8.21 s
Wall time: 8.21 s


In [397]:
df['승차-배차'] = 승차배차_val

### '승차-예정'

In [398]:
%%time

승차예정_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['예정시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['승차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    승차예정_val.append(time_interval)

CPU times: user 8.08 s, sys: 13.9 ms, total: 8.1 s
Wall time: 8.1 s


In [399]:
df['승차-예정'] = 승차예정_val

In [400]:
df.head()

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정,승차-배차,승차-예정
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,-1 days +23:47:06,0 days 00:00:33,-1 days +23:47:39
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53


예상시간보다 일찍 도착하면, 시간차이 구할때 -1day씩 추가계산되는 문제가 있다.

### -1 days 점검

In [401]:
%%time

배차예정2 = []
for i in range(len(df)):
    배차예정2.append(str(df['배차-예정'][i]))

CPU times: user 4.73 s, sys: 15.4 ms, total: 4.74 s
Wall time: 4.74 s


In [402]:
df['배차-예정test']=배차예정2

In [403]:
df[df['배차-예정test'].str.contains('-1 days')] ## 407,553 중 37,987행 존재 -> 약 9.3% 정도 일찍 배차됨

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정,승차-배차,승차-예정,배차-예정test
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,-1 days +23:47:06,0 days 00:00:33,-1 days +23:47:39,-1 days +23:47:06
25,9568,중형승합,2018-01-23 04:30:00,2018-01-23 04:00:59,2018-01-23 04:47:02,구로구,오류제1동,영등포구,대림제3동,-1 days +23:30:59,0 days 00:46:03,0 days 00:17:02,-1 days +23:30:59
27,7819,중형승합,2018-01-23 04:40:00,2018-01-23 04:02:59,2018-01-23 04:43:09,종로구,부암동,종로구,창신제1동,-1 days +23:22:59,0 days 00:40:10,0 days 00:03:09,-1 days +23:22:59
28,1061,중형승합,2018-01-23 04:40:00,2018-01-23 03:57:30,2018-01-23 04:44:27,송파구,장지동,강동구,명일제1동,-1 days +23:17:30,0 days 00:46:57,0 days 00:04:27,-1 days +23:17:30
30,5804,중형승합,2018-01-23 05:00:00,2018-01-23 04:32:36,2018-01-23 04:56:19,성북구,종암동,종로구,청운효자동,-1 days +23:32:36,0 days 00:23:43,-1 days +23:56:19,-1 days +23:32:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
407506,5829,중형승용,2022-06-24 07:00:00,2022-06-24 06:35:46,2022-06-24 08:12:29,성동구,마장동,서초구,서초1동,-1 days +23:35:46,0 days 01:36:43,0 days 01:12:29,-1 days +23:35:46
407509,8143,중형승합,2022-06-24 07:00:00,2022-06-24 06:19:36,2022-06-24 07:25:45,관악구,낙성대동,서초구,반포4동,-1 days +23:19:36,0 days 01:06:09,0 days 00:25:45,-1 days +23:19:36
407511,8167,중형승합,2022-06-24 07:00:00,2022-06-24 06:40:43,2022-06-24 07:20:49,강서구,화곡제4동,구로구,수궁동,-1 days +23:40:43,0 days 00:40:06,0 days 00:20:49,-1 days +23:40:43
407523,1921,중형승용,2022-06-24 07:00:00,2022-06-24 06:28:40,2022-06-24 07:30:24,강서구,화곡본동,강남구,청담동,-1 days +23:28:40,0 days 01:01:44,0 days 00:30:24,-1 days +23:28:40


-----------------------

In [404]:
# for문전에 돌리기
df["예정시간"]=pd.to_datetime(df["예정시간"])
df["배차시간"]=pd.to_datetime(df["배차시간"])
df["승차시간"]=pd.to_datetime(df["승차시간"])

In [405]:
# test1

배차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_3 = time_2 + datetime.timedelta(hours=23, minutes=60, seconds=60) # datetime + timedelta 가능
        time_interval = time_3 - time_1
        배차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        배차예정_val.append(time_interval)

In [406]:
# test2 채택★- 순시간구하기 -> 일찍 도착한 셀 마이너스 붙이기 전!

배차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        배차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        배차예정_val.append(time_interval)

승차배차_val = []
for i in range(len(df)):
    time_1 = df['배차시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        승차배차_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        승차배차_val.append(time_interval)
        
승차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        승차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        승차예정_val.append(time_interval)

In [407]:
df['배차-예정'] = 배차예정_val 
df['승차-배차'] = 승차배차_val 
df['승차-예정'] = 승차예정_val 

In [408]:
df

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정,승차-배차,승차-예정,배차-예정test
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
407548,8579,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38
407549,8006,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10
407550,8261,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27
407551,1558,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,0 days 01:06:22,0 days 01:11:21,0 days 00:04:59,0 days 01:06:22


0days 00:00:00 형식에서 분단위에 마이너스 붙이는 방법 누가알려줘라 

### 시간차이 칼럼 분단위 통합

In [409]:
#test- 분변환
duration = datetime.timedelta(days = 0, hours=2, minutes=30, seconds=60)
seconds = duration.total_seconds()
minutes = seconds//60

minutes

151.0

In [410]:
# 반복 
배차예정_val = []

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        배차예정_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        배차예정_val.append(minutes)

In [411]:
df['배차-예정(분)'] = 배차예정_val

In [412]:
df

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정,승차-배차,승차-예정,배차-예정test,배차-예정(분)
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42,14.0
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37,33.0
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06,-13.0
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19,4.0
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407548,8579,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38,12.0
407549,8006,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10,9.0
407550,8261,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27,4.0
407551,1558,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,0 days 01:06:22,0 days 01:11:21,0 days 00:04:59,0 days 01:06:22,66.0


'배차-예정' 마이너스 값 + 분단위 통합 성공 -> 이제 나머지 '승차-배차', '승차-예정'도 통합시켜주자

In [413]:
%%time

승차배차_val = []

for i in range(len(df)):
    time_1 = df['배차시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        승차배차_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        승차배차_val.append(minutes)


CPU times: user 9.07 s, sys: 13 ms, total: 9.08 s
Wall time: 9.08 s


In [414]:
df['승차-배차(분)'] = 승차배차_val

In [415]:
%%time

승차예정_val = []

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        승차예정_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        승차예정_val.append(minutes)

CPU times: user 9.14 s, sys: 24.3 ms, total: 9.16 s
Wall time: 9.16 s


In [416]:
df['승차-예정(분)'] = 승차예정_val

In [417]:
df

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정,승차-배차,승차-예정,배차-예정test,배차-예정(분),승차-배차(분),승차-예정(분)
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42,14.0,15.0,30.0
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37,33.0,28.0,62.0
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06,-13.0,0.0,-13.0
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19,4.0,18.0,22.0
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17,36.0,22.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407548,8579,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38,12.0,42.0,55.0
407549,8006,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10,9.0,16.0,25.0
407550,8261,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27,4.0,21.0,25.0
407551,1558,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,0 days 01:06:22,0 days 01:11:21,0 days 00:04:59,0 days 01:06:22,66.0,-72.0,-5.0


In [418]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407553 entries, 0 to 407552
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   no         407553 non-null  int64          
 1   cartype    407553 non-null  object         
 2   예정시간       407553 non-null  datetime64[ns] 
 3   배차시간       407553 non-null  datetime64[ns] 
 4   승차시간       407553 non-null  datetime64[ns] 
 5   startpos1  407553 non-null  object         
 6   startpos2  407553 non-null  object         
 7   endpos1    407553 non-null  object         
 8   endpos2    407553 non-null  object         
 9   배차-예정      407553 non-null  timedelta64[ns]
 10  승차-배차      407553 non-null  timedelta64[ns]
 11  승차-예정      407553 non-null  timedelta64[ns]
 12  배차-예정test  407553 non-null  object         
 13  배차-예정(분)   407553 non-null  float64        
 14  승차-배차(분)   407553 non-null  float64        
 15  승차-예정(분)   407553 non-null  float64        
dtypes:

In [419]:
df.drop(['배차-예정','승차-배차','승차-예정','배차-예정test'], axis=1, inplace=True)

In [420]:
df

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,배차-예정(분),승차-배차(분),승차-예정(분)
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,14.0,15.0,30.0
1,1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,33.0,28.0,62.0
2,5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,-13.0,0.0,-13.0
3,7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,4.0,18.0,22.0
4,1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,36.0,22.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...
407548,8579,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,12.0,42.0,55.0
407549,8006,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,9.0,16.0,25.0
407550,8261,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,4.0,21.0,25.0
407551,1558,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,66.0,-72.0,-5.0


In [421]:
df.to_csv('master2_timediff.csv', index = False, encoding = 'utf-8')

 ## 2. 좌표(위도,경도) 칼럼생성

## 3. 좌표간 거리 칼럼 생성